In [126]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import xarray as xr
import sparse
from itertools import product, combinations
from pathlib import Path
from pprint import pprint

pd.options.display.max_rows = 4000
#pd.options.display.max_columns = 4000

import sys
sys.path.append('../src/')

import dimensions as dims
import categories as cats
import constraints

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Finding the seed distribution from the extended questionaire sample

## Find hidden zero-cells with non-zero constraints/Complete the set of marginals

In [2]:
# Find set of constraints sharing dimensions

# Extract set of dimensions
for cname, cdict in constraints.constraints_ind.items():
    if isinstance(cdict, list):
        print(cname)

In [3]:
personas = pd.read_csv(
        Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/') / 'Personas19.CSV')#.drop(columns=cols_drop)
personas_cat = cats.process_people_df(personas)

### Coding the extended survey

In [159]:
col_sets = {}

for const_name, const_dict in constraints.constraints_ind.items():
    num_cols = len(const_dict)
    for col_name, col_classes in const_dict.items():
        if col_name not in col_sets:
            col_sets[col_name] = {'constraints': {const_name: const_dict}, 'classes': set(col_classes), 'num_cols': num_cols}
        else:
            if num_cols == col_sets[col_name]['num_cols']:
                col_sets[col_name]['constraints'][const_name] = const_dict
                col_sets[col_name]['classes'].update(col_classes)
            elif num_cols < col_sets[col_name]['num_cols']:
                col_sets[col_name] = {'constraints': {const_name, const_dict}, 'classes': set(col_classes), 'num_cols': num_cols}

constrained_cols = list(col_sets.keys())

missing_marg_dict = {}
for col in constrained_cols:
    full_cat_set = set(personas_cat[col].cat.categories)
    missing_marginals = full_cat_set - col_sets[col]['classes']
    #if len(missing_marginals) == 0:
    #    continue
    
    missing_marg_dict[col] = missing_marginals

for col, missing in  missing_marg_dict.items():
    col_sets[col]['missing'] = missing

In [160]:
pd.DataFrame(col_sets)

,SEXO,EDAD,ENT_PAIS_NAC,ENT_PAIS_RES_5A,HLENGUA,AFRODES,DIS,ASISTEN,ALFABET,NIVACAD,CONACT,DHSERSAL,SITUA_CONYUGAL,RELIGION
constraints,"{'POBFEM': {'SEXO': ['F']}, 'POBMAS': {'SEXO':...","{'P_0A2': {'EDAD': ['0-2']}, 'P_3YMAS': {'EDAD...","{'PNACENT': {'ENT_PAIS_NAC': ['EstaEnt']}, 'PN...","{'PRES2015': {'EDAD': ['5', '6-7', '8-11', '12...","{'P3YM_HLI': {'EDAD': ['3-4', '5', '6-7', '8-1...",{'POB_AFRO': {'AFRODES': ['Sí']}},"{'PCON_DISC': {'DIS': ['1111135', '1111136', '...","{'P3A5_NOA': {'EDAD': ['3-4', '5'], 'ASISTEN':...","{'P8A14AN': {'EDAD': ['8-11', '12-14'], 'ALFAB...","{'P15YM_SE': {'EDAD': ['15-17', '18-24', '25-4...","{'PEA': {'EDAD': ['12-14', '15-17', '18-24', '...","{'PSINDER': {'DHSERSAL': ['0000000010']}, 'PDE...","{'P12YM_SOLT': {'EDAD': ['12-14', '15-17', '18...","{'PCATOLICA': {'RELIGION': ['Católica']}, 'PRO..."
classes,"{F, M}","{0-2, 60-64, 3-4, 5, 65-130, 12-14, 8-11, 18-2...","{EstaEnt, OtraEnt}","{EstaEnt, OtraEnt}","{Sí/Sí Español, Sí/No especificado, Sí/No espa...",{Sí},"{2832835, 3283829, 1242249, 9399325, 8119236, ...","{No, Sí}",{No},"{Especialidad_1, Especialidad_2, Normal de lic...","{Trabajó, No se tiene información / se rescata...","{1000100000, 0010000100, 0001001000, 101000000...","{casada(o) sólo religiosamente, soltera(o), ca...","{Protestante/cristiano evangélico, Católica, O..."
num_cols,1,1,1,2,2,1,1,2,2,2,2,1,2,1
missing,{},{Unknown},"{No especificado, Blanco por pase, OtroPais}","{No especificado, Blanco por pase, OtroPais}","{No especificado, Blanco por pase, No}","{No especificado, No}","{9981916, 1889886, 8998999, 8118196, 9881119, ...","{No especificado, Blanco por pase}","{No especificado, Blanco por pase, Sí}","{No especificado, Blanco por pase}","{No especificado, Blanco por pase}",{0000000001},"{No especificado, Blanco por pase}",{Religión no especificada}


In [226]:
len(col_sets[col]['classes'])

12

In [ ]:
from math import comb

# Find groups of disjoint constraints
col = 'EDAD'
consts = col_sets[col]['constraints']
comb_list = []
matches_obs = False
for k in range(1, 16):
    for c in combinations(consts, k):
        len_union = len(set([ccc for cc in c for ccc in consts[cc][col]]))
        sum_lens = sum([len(consts[cc][col]) for cc in c])
        if len_union == sum_lens:
            comb_list.append([len_union, -len(c), c, len_union == len(col_sets[col]['classes'])])
            if len_union == len(col_sets[col]['classes']):
                matches_obs = True
        
sorted(comb_list, reverse=True)

In [11]:
personas_agg = cats.build_X(personas_cat)

In [18]:
%%time
XW_dict = cats.process_extended_survey(Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/'), verbose=False)

CPU times: user 1min 24s, sys: 854 ms, total: 1min 25s
Wall time: 1min 25s


In [409]:
df_mun, df_loc, df_agebs = cats.process_census(
    Path('../data/census_loc/'),
    Path('../data/census_ageb_manz/'))

In [412]:
df_agebs_min = df_agebs.replace(-1, 0)
df_agebs_max = df_agebs.replace(-1, 2)

In [420]:
for df in [df_agebs, df_agebs_min, df_agebs_max]:
    df['P_UNK'] = df.POBTOT - (df.POB0_14 + df.P_15YMAS)

In [390]:
np.nan != np.nan

True

In [392]:
np.nan == np.nan

False

In [431]:
pd.concat(
    [
        df_agebs_min.P_UNK,
        df_agebs_max.P_UNK, 
        df_agebs.P_UNK,
        df_agebs['P_UNK'].where(
            np.logical_or(
                df_agebs_min['P_UNK'] == df_agebs_max['P_UNK'],
                df_agebs['P_UNK'].isna()
            ),
            -1
        )
    ],
    axis=1)

P_UNK   P_UNK   P_UNK   P_UNK
MUN LOC AGEB                                
1   1   0021     2.0     2.0     2.0     2.0
        0036     0.0     0.0     0.0     0.0
        0040     0.0     0.0     0.0     0.0
        0055    10.0     6.0    12.0    -1.0
2   1   0207     0.0     0.0     0.0     0.0
        0211     0.0     0.0     0.0     0.0
        0226     0.0     0.0     0.0     0.0
        0230     0.0     0.0     0.0     0.0
        0245     0.0     0.0     0.0     0.0
        025A     0.0     0.0     0.0     0.0
        0264     0.0     0.0     0.0     0.0
        0279     0.0     0.0     0.0     0.0
        0438     0.0     0.0     0.0     0.0
        0442     0.0     0.0     0.0     0.0
        0457     2.0     0.0     3.0    -1.0
3   1   0083     0.0     0.0     0.0     0.0
        0098     0.0     0.0     0.0     0.0
        0100     0.0     0.0     0.0     0.0
        0115     0.0     0.0     0.0     0.0
        0204     8.0     4.0    10.0    -1.0
        0219     4.0     0.0     6.0    -1.0
        0238     5.0     1.0     7.0    -1.0
4   1   0095     0.0     0.0     0.0     0.0
        0112     0.0     0.0     0.0     0.0
        0127     0.0     0.0     0.0     0.0
        0131     0.0     0.0     0.0     0.0
        0146     0.0     0.0     0.0     0.0
        0150     0.0     0.0     0.0     0.0
        0165     0.0     0.0     0.0     0.0
        0184     0.0     0.0     0.0     0.0
        0216     6.0     6.0     6.0     6.0
        0220     0.0     0.0     0.0     0.0
        0235     3.0     3.0     3.0     3.0
        024A     0.0     0.0     0.0     0.0
        0254     0.0     0.0     0.0     0.0
        0269     0.0     0.0     0.0     0.0
        0273     0.0     0.0     0.0     0.0
        0288     0.0     0.0     0.0     0.0
        0292     0.0     0.0     0.0     0.0
        0305     0.0     0.0     0.0     0.0
        031A     0.0     0.0     0.0     0.0
        0324     0.0     0.0     0.0     0.0
        0339     0.0     0.0     0.0     0.0
        0343     0.0     0.0     0.0     0.0
        0358     0.0     0.0     0.0     0.0
        0362     0.0     0.0     0.0     0.0
        0377     3.0     3.0     3.0     3.0
        0381     0.0     0.0     0.0     0.0
        0413     0.0     0.0     0.0     0.0
        0428     0.0     0.0     0.0     0.0
        0432     0.0     0.0     0.0     0.0
        0447     0.0     0.0     0.0     0.0
        0466     5.0     1.0     7.0    -1.0
        0470     0.0     0.0     0.0     0.0
        0485     0.0     0.0     0.0     0.0
5   1   0406     0.0     0.0     0.0     0.0
        0410     0.0     0.0     0.0     0.0
        0482     0.0     0.0     0.0     0.0
        0497     0.0     0.0     0.0     0.0
        050A     0.0     0.0     0.0     0.0
        0514     0.0     0.0     0.0     0.0
        0529     0.0     0.0     0.0     0.0
        0533     0.0     0.0     0.0     0.0
        0548     0.0     0.0     0.0     0.0
        0552     0.0     0.0     0.0     0.0
        0567     0.0     0.0     0.0     0.0
        0571     0.0     0.0     0.0     0.0
        0586     0.0     0.0     0.0     0.0
        0980     1.0    -1.0     2.0    -1.0
        1048     0.0     0.0     0.0     0.0
        1052     0.0     0.0     0.0     0.0
        1071     0.0     0.0     0.0     0.0
        1086     1.0    -1.0     2.0    -1.0
        1090     0.0     0.0     0.0     0.0
        1103     0.0     0.0     0.0     0.0
        1118     NaN     NaN     NaN     NaN
        1137     1.0    -3.0     3.0    -1.0
        1141     0.0     0.0     0.0     0.0
        1160     0.0     0.0     0.0     0.0
        1175     0.0     0.0     0.0     0.0
        118A     NaN     NaN     NaN     NaN
6   1   0032     0.0     0.0     0.0     0.0
        0051     4.0     4.0     4.0     4.0
        0066     1.0     1.0     1.0     1.0
        0070     0.0     0.0     0.0     0.0
        0085     0.0     0.0     0.0     0.0
        009A     0.0     0.0     0.0     0.0
        01

In [382]:
(df_agebs == -1).sum().sum()

42588

In [383]:
np.logical_and(df_agebs_min != df_agebs_max, ~df_agebs.isna()).sum().sum()

42588

In [189]:
X, W = (*XW_dict['Abasolo'].values(),)
X = X.copy()
W = W.copy()
C = df_mun.loc[1][W.index].copy()

In [190]:
WC = W.copy()
WC['C'] = C

In [192]:
W.shape[0], np.linalg.matrix_rank(W), np.linalg.matrix_rank(WC)

(157, 93, 94)

In [206]:
# Try obtain a matrix with full row rank
for c in ['_'.join(c.split('_')[:-1]) for c in C.index if c.endswith('_M')]:
    assert C[c] == C[c + '_M'] + C[c + '_F'], c
    W = W.drop(c)
    C = C.drop(c)

In [208]:
W.shape[0], np.linalg.matrix_rank(W), np.linalg.matrix_rank(WC)

(125, 93, 94)

In [209]:
C

POBTOT         2974.0
POBFEM         1532.0
POBMAS         1442.0
P_0A2_F          88.0
P_0A2_M          77.0
P_3YMAS_M      1364.0
P_3YMAS_F      1443.0
P_5YMAS_M      1317.0
P_5YMAS_F      1380.0
P_12YMAS_M     1134.0
P_12YMAS_F     1193.0
P_15YMAS_M     1060.0
P_15YMAS_F     1115.0
P_18YMAS_M      981.0
P_18YMAS_F     1026.0
P_3A5_M          75.0
P_3A5_F          91.0
P_6A11_M        155.0
P_6A11_F        159.0
P_8A14_M        185.0
P_8A14_F        183.0
P_12A14_M        74.0
P_12A14_F        78.0
P_15A17_M        79.0
P_15A17_F        89.0
P_18A24_M       165.0
P_18A24_F       193.0
P_15A49_F       779.0
P_60YMAS_M      155.0
P_60YMAS_F      173.0
POB0_14         797.0
POB15_64       1959.0
POB65_MAS       216.0
PNACENT_F      1261.0
PNACENT_M      1214.0
PNACOE_F        250.0
PNACOE_M        217.0
PRES2015_F     1343.0
PRES2015_M     1295.0
PRESOE15_F       27.0
PRESOE15_M       18.0
P3YM_HLI_F        6.0
P3YM_HLI_M        6.0
P3HLINHE_F        0.0
P3HLINHE_M        0.0
P3HLI_HE_F

In [187]:
C.P12YM_SOLT + C.P12YM_CASA + C.P12YM_SEPA 

2322.0

In [188]:
C.P_12YMAS

2327.0

In [162]:
X = X.FACTOR.values
W = W.values
C = C.values

In [163]:
W = W[1:]

In [164]:
C = C[1:]

In [78]:
import scipy.linalg as spl

In [167]:
W_inv = spl.pinv(W)
X_sol = W_inv @ C

In [168]:
np.linalg.norm(W @ X - C)

101.02474944289642

In [172]:
np.linalg.norm(W @ X_sol - C)

3.8349685746458633

In [27]:
# The following constraints cannot be satisfied from the seed distribution unless they are 0
W.T.sum()[W.T.sum() == 0]

P3HLINHE      0
P3HLINHE_F    0
P3HLINHE_M    0
P5_HLI_NHE    0
P8A14AN_F     0
PAFIL_PDOM    0
PDER_IMSSB    0
POTRAS_REL    0
dtype: int64

### OD Survey

In [96]:
od_df = pd.read_csv('../data/OD_Survey/datos_limpios_tiempos.csv', low_memory=False)

In [101]:
od_df.head()

,ID-HOGAR,H-P,H-P-V,Latitud,Longitud,FechaHoraEnc,NumVisita,TipoEnc,RealizoEnc,Encuestador,Supervisor,Cod_MunDomicilio,ColDom,CalleDom,NExtDom,NIntDom,RefDom,CPDom,Punto_zona,Cod_EdoDomicilio,LineaTelef,VHAuto,VHMoto,VHPickup,VHCamion,VHBici,VHPatineta,VHPatines,VHScooter,VHOtro,CHBaños,CHDormitorios,Internet,Hab14masTrabajo,HabitantesTotal,HbitantesMayor6,HbitantesMenor5,TodosEstan,NunHabitante,Género,Edad,RelaciónHogar,RelaciónHogar_O,Discapacidad,Discapacidad_O,Estudios,Estudios_O,Ocupacion,Ocupacion_O,SectorEconom,SectorEconom_O,ViajeAyer,Num_Viaje,Lugar_Or,Cod_MunOri,ColOri,RefOri,CalleOri,Esquina_Ori,Cruce_Ori,OtroEstadoOri,OtroEstadoOri_O,CodOri,Cod_EdoOri,Macrozona Origen,ZonaOri,Hora Inicio V,LugarDest,Cod_MunDest,ColDest,RefDest,CalleDest,Esquina_Dest,Cruce_Dest,OtroEstadoDest,OtroEstadoDest_O,CodDest,Cod_EdoDest,Macrozona Destino,ZonaDest,Hora Término Viaje,Cod_IDEdoDest,Cod_IDMunDest,Cod_IDLocDest,Cod_LocDest,Cod_IDColDest,Cod_ColDest,Cod_IDRefDest,Cod_RefDest,Tiempo Tot de Viaje,Motivo,Motivo_O,M1_TipoTransp,M1_Transp_O,M1Tpo_Caminata,M1N_Ruta,M1_HHTpoParada,M1_MMTpoParada,M1_HHTpoAbordo,M1_HHTpoAbordo_O,M1_MMTpoAbordo,M1_Pago,M2_Transp,M2_Transp_O,M2_TipoTransp,M2Tpo_Caminata,M2N_Ruta,M2_HHTpoParada,M2_MMTpoParada,M2_TpoTranspordo,M2_HHTpoAbordo,M2_HHTpoAbordo_O,M2_MMTpoAbordo,M2_Pago,M3_Transp,M3_Transp_O,M3_TipoTransp,M3Tpo_Caminata,M3N_Ruta,M3_HHTpoParada,M3_MMTpoParada,M3_TpoTranspordo,M3_HHTpoAbordo,M3_HHTpoAbordo_O,M3_MMTpoAbordo,M3_Pago,M4_Transp,M4_Transp_O,M4_TipoTransp,M4Tpo_Caminata,M4N_Ruta,M4_HHTpoParada,M4_MMTpoParada,M4_TpoTranspordo,M4_HHTpoAbordo,M4_HHTpoAbordo_O,M4_MMTpoAbordo,M4_Pago,M5_Transp,M5_Transp_O,M5_TipoTransp,M5Tpo_Caminata,M5N_Ruta,M5_HHTpoParada,M5_MMTpoParada,M5_TpoTranspordo,M5_HHTpoAbordo,M5_HHTpoAbordo_O,M5_MMTpoAbordo,M5_Pago,M6_Transp,M6_Transp_O,M6_TipoTransp,M6Tpo_Caminata,M6N_Ruta,M6_HHTpoParada,M6_MMTpoParada,M6_TpoTranspordo,M6_HHTpoAbordo,M6_HHTpoAbordo_O,M6_MMTpoAbordo,M6_Pago,M7_Transp,M7_Transp_O,M7_TipoTransp,M7Tpo_Caminata,M7N_Ruta,M7_HHTpoParada,M7_MMTpoParada,M7_TpoTranspordo,M7_HHTpoAbordo,M7_HHTpoAbordo_O,M7_MMTpoAbordo,M7_Pago,M8_Transp,M8_Transp_O,M8_TipoTransp,M8Tpo_Caminata,M8N_Ruta,M8_HHTpoParada,M8_MMTpoParada,M8_TpoTranspordo,M8_HHTpoAbordo,M8_HHTpoAbordo_O,M8_MMTpoAbordo,M8_Pago,TipoEstacionamiento,TpoBusqueda,TpoEstacionadoHH,TpoEstacionadoMM,CostoEstacionamiento,Obs_Encuestador,Obs_ENCUESTA,FE,Modo Agrupado,motivos,genero,estudios,disc,origen,Tiempo,tiempo_s,tiempo_m,tiempo_h
0,23853a-20,23853a-20/2,23853a-20/2-2,25.765088,-100.406374,2019-09-24T00:00:00Z,Visita 1,HABITANTE,NaN,Lucia Hernández Mondragón,Patricia Baena,Monterrey,Barrio Estrella Elit,lucero,112,NaN,NaN,64102.0,86,Nuevo Leon,Si,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,Si,4.0,4.0,4.0,NaN,NaN,2,Mujer,40,Madre/Esposa,NaN,Ninguna,NaN,Sin Instrucción,NaN,Ama de casa,NaN,Otro,NaN,Sí,2,El DESTINO de viaje inmediato anterior,Monterrey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10358.0,Nuevo Leon,19.0,76,1899-12-31T18:50:00Z,El HOGAR,Monterrey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2249.0,Nuevo Leon,17.0,86,1899-12-31T19:00:00Z,19.0,39.0,NaN,NaN,34979,BARRIO ESTRELLA,NaN,NaN,1899-12-31T00:10:00Z,regreso a casa,NaN,Vehículo Particular,NaN,0.0,NaN,0.0,0,0,0.0,10,NaN,No utilizó otro medio de transporte,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,"Propio (Casa, oficina)",NaN,11.0,NaN,NaN,NaN,NaN,1282.883061,automovil,3,F,Bajo,No,NA+NA,NaN,600,10,0.166667
1,23853a-20,23853a-20/4,23853a-20/4-2,25.765088,-100.406374,2019-09-24T00:00:00Z,Visita 1,VIAJE,NaN,Lucia Hernández Mondragón,Patricia Baena,Monterrey,Barrio Estrella Elit,lucero,112,NaN,NaN,64102.0,86,Nuevo Leon,Si,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,Si,4.0,4.0,4.0,NaN,NaN,4,Hombre,

### Determining household structure

In [147]:
# Map people to households
viv_per_dict = {}
for id_viv, group in personas.groupby('ID_VIV'):
    viv_per_dict[id_viv] = group[['ID_PERSONA', 'NUMPER', 'SEXO', 'EDAD', 'PARENTESCO', 'IDENT_MADRE', 'IDENT_PADRE']]

In [148]:
viv_per_dict[190010000001].sort_values('NUMPER')

,ID_PERSONA,NUMPER,SEXO,EDAD,PARENTESCO,IDENT_MADRE,IDENT_PADRE
4,19001000000100005,1,3,30,101,96,96
1,19001000000100002,2,1,36,201,96,96
0,19001000000100001,3,3,9,301,1,96
3,19001000000100004,4,3,6,301,1,96
2,19001000000100003,5,1,1,301,1,2


### Seed from group counts

In [ ]:
# Seed for whole met area

In [ ]:
# Seed per municipality

In [ ]:
# Comparing seeds from municipality and whole area

### Seed using bayesian networks

### Comparing both seeds

# Reconstrucción de microdatos para el censo 2020, nivel AGEB

El objetivo es obtener una base de microdatos para cada AGEB del censo 2020 consistente con los tabulados oficiales.
Para tal efecto, vamos a probar diferentes metodologías, comenzando con una estrategia de reconstrucción utilizando métodos de programación lineal.

Para tener una idea clara de la implementación, tomemos una AGEB específica de la zona metropolitana de Monterrey.

A nivel municipio o demarcación territorial, localidad y AGEB, cualquier indicador con menos de tres unidades aparece con asterisco
a excepción de las variables Población total (POBTOT), Total de viviendas (VIVTOT) y Total de viviendas habitadas (TVIVHAB).

In [25]:
personas_cat.FACTOR.sum()

5768781

In [24]:
df_entidad

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,AGEB,MZA,POBTOT,POBFEM,POBMAS,P_0A2,P_0A2_F,P_0A2_M,P_3YMAS,P_3YMAS_F,P_3YMAS_M,P_5YMAS,P_5YMAS_F,P_5YMAS_M,P_12YMAS,P_12YMAS_F,P_12YMAS_M,P_15YMAS,P_15YMAS_F,P_15YMAS_M,P_18YMAS,P_18YMAS_F,P_18YMAS_M,P_3A5,P_3A5_F,P_3A5_M,P_6A11,P_6A11_F,P_6A11_M,P_8A14,P_8A14_F,P_8A14_M,P_12A14,P_12A14_F,P_12A14_M,P_15A17,P_15A17_F,P_15A17_M,P_18A24,P_18A24_F,P_18A24_M,P_15A49_F,P_60YMAS,P_60YMAS_F,P_60YMAS_M,REL_H_M,POB0_14,POB15_64,POB65_MAS,PROM_HNV,PNACENT,PNACENT_F,PNACENT_M,PNACOE,PNACOE_F,PNACOE_M,PRES2015,PRES2015_F,PRES2015_M,PRESOE15,PRESOE15_F,PRESOE15_M,P3YM_HLI,P3YM_HLI_F,P3YM_HLI_M,P3HLINHE,P3HLINHE_F,P3HLINHE_M,P3HLI_HE,P3HLI_HE_F,P3HLI_HE_M,P5_HLI,P5_HLI_NHE,P5_HLI_HE,PHOG_IND,POB_AFRO,POB_AFRO_F,POB_AFRO_M,PCON_DISC,PCDISC_MOT,PCDISC_VIS,PCDISC_LENG,PCDISC_AUD,PCDISC_MOT2,PCDISC_MEN,PCON_LIMI,PCLIM_CSB,PCLIM_VIS,PCLIM_HACO,PCLIM_OAUD,PCLIM_MOT2,PCLIM_RE_CO,PCLIM_PMEN,PSIND_LIM,P3A5_NOA,P3A5_NOA_F,P3A5_NOA_M,P6A11_NOA,P6A11_NOAF,P6A11_NOAM,P12A14NOA,P12A14NOAF,P12A14NOAM,P15A17A,P15A17A_F,P15A17A_M,P18A24A,P18A24A_F,P18A24A_M,P8A14AN,P8A14AN_F,P8A14AN_M,P15YM_AN,P15YM_AN_F,P15YM_AN_M,P15YM_SE,P15YM_SE_F,P15YM_SE_M,P15PRI_IN,P15PRI_INF,P15PRI_INM,P15PRI_CO,P15PRI_COF,P15PRI_COM,P15SEC_IN,P15SEC_INF,P15SEC_INM,P15SEC_CO,P15SEC_COF,P15SEC_COM,P18YM_PB,P18YM_PB_F,P18YM_PB_M,GRAPROES,GRAPROES_F,GRAPROES_M,PEA,PEA_F,PEA_M,PE_INAC,PE_INAC_F,PE_INAC_M,POCUPADA,POCUPADA_F,POCUPADA_M,PDESOCUP,PDESOCUP_F,PDESOCUP_M,PSINDER,PDER_SS,PDER_IMSS,PDER_ISTE,PDER_ISTEE,PAFIL_PDOM,PDER_SEGP,PDER_IMSSB,PAFIL_IPRIV,PAFIL_OTRAI,P12YM_SOLT,P12YM_CASA,P12YM_SEPA,PCATOLICA,PRO_CRIEVA,POTRAS_REL,PSIN_RELIG,TOTHOG,HOGJEF_F,HOGJEF_M,POBHOG,PHOGJEF_F,PHOGJEF_M,VIVTOT,TVIVHAB,TVIVPAR,VIVPAR_HAB,VIVPARH_CV,TVIVPARHAB,VIVPAR_DES,VIVPAR_UT,OCUPVIVPAR,PROM_OCUP,PRO_OCUP_C,VPH_PISODT,VPH_PISOTI,VPH_1DOR,VPH_2YMASD,VPH_1CUART,VPH_2CUART,VPH_3YMASC,VPH_C_ELEC,VPH_S_ELEC,VPH_AGUADV,VPH_AEASP,VPH_AGUAFV,VPH_TINACO,VPH_CISTER,VPH_EXCSA,VPH_LETR,VPH_DRENAJ,VPH_NODREN,VPH_C_SERV,VPH_NDEAED,VPH_DSADMA,VPH_NDACMM,VPH_SNBIEN,VPH_REFRI,VPH_LAVAD,VPH_HMICRO,VPH_AUTOM,VPH_MOTO,VPH_BICI,VPH_RADIO,VPH_TV,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC
0,19,Nuevo León,0,Total de la entidad Nuevo León,0,Total de la entidad,0000,0,5784442,2893492.0,2890950.0,258166.0,127997.0,130169.0,5508144.0,2756412.0,2751732.0,5311401.0,2659307.0,2652094.0,4659636.0,2339039.0,2320597.0,4384862.0,2203851.0,2181011.0,4111381.0,2070207.0,2041174.0,294171.0,145102.0,149069.0,554337.0,272271.0,282066.0,646846.0,318017.0,328829.0,274774.0,135188.0,139586.0,273481.0,133644.0,139837.0,702241.0,346304.0,355937.0,1549927.0,654050.0,350903.0,303147.0,99.91,1381448.0,3948050.0,436812.0,1.88,4381647.0,2192985.0,2188662.0,1319964.0,661997.0,657967.0,4993000.0,2509179.0,2483821.0,277369.0,133363.0,144006.0,77945.0,37686.0,40259.0,325.0,218.0,107.0,76296.0,36790.0,39506.0,77618.0,312.0,76007.0,159873.0,97603.0,47731.0,49872.0,220206.0,103024.0,94260.0,37352.0,43164.0,50468.0,39164.0,551374.0,151424.0,369318.0,34152.0,103774.0,26256.0,85652.0,69219.0,4950186.0,109729.0,53790.0,55939.0,22177.0,10617.0,11560.0,18183.0,8456.0,9727.0,198445.0,97890.0,100555.0,231722.0,113905.0,117817.0,10547.0,4326.0,6221.0,63980.0,34933.0,29047.0,87239.0,46646.0,40593.0,204645.0,110270.0,94375.0,430136.0,235652.0,194484.0,104046.0,46671.0,57375.0,1236867.0,619561.0,617306.0,2145139.0,1048339.0,1096800.0,10.74,10.6,10.88,2862003.0,1111505.0,1750498.0,1779940.0,1220647.0,559293.0,2807449.0,1096078.0,1711371.0,54554.0,15427.0,39127.0,1072664.0,4681465.0,3596116.0,178747.0,41527.0,28744.0,558212.0,10068.0,330130.0,86910.0,1542048.0,2599029.0,509936.0,4492947.0,690097.0,5094.0,561584.0,1655256.0,451544.0,1203712.0,5768759.0,1448464.0,4320295.0,2037695,1655690.0,1927423.0,1545418.0,1654199.0,1655256.0,286185.0,95820.0,5768759.0,3.49,0.85,1634277.0,13939.0,398158.0,1250292.0,27975.0,145232.0,1475200.0,1645742.0,2849.0,1636431.0,1596634

In [317]:
# Load census data
df_censo = pd.read_csv('../data/census_ageb_manz/RESAGEBURB_19CSV20.csv', na_values=['*', 'N/D'], low_memory=False)

# Filter for agebs
df_entidad = df_censo.query('ENTIDAD != 0 & MUN == 0 & LOC == 0 & AGEB == "0000" & MZA == 0')
df_mun = df_censo.query('ENTIDAD != 0 & MUN != 0 & LOC == 0 & AGEB == "0000" & MZA == 0')
df_loc = df_censo.query('ENTIDAD != 0 & MUN != 0 & LOC != 0 & AGEB == "0000" & MZA == 0')
df_agebs = df_censo.query('ENTIDAD != 0 & MUN != 0 & LOC != 0 & AGEB != "0000" & MZA == 0')

# Eliminar AGEBS sin población
df_agebs = df_agebs[df_agebs.POBTOT > 0]

# Hay situaciones en que sólo uno de POBMAS Y POBFEM se omiten, en cuyo caso se puede imputar el valor
#POBMAS_not_nan = ~df_agebs.POBMAS.isna()
#POBFEM_not_nan = ~df_agebs.POBFEM.isna()
#df_agebs.loc[POBMAS_not_nan, 'POBFEM'] = df_agebs.loc[POBMAS_not_nan, 'POBTOT'] - df_agebs.loc[POBMAS_not_nan, 'POBMAS']
#df_agebs.loc[POBFEM_not_nan, 'POBMAS'] = df_agebs.loc[POBFEM_not_nan, 'POBTOT'] - df_agebs.loc[POBFEM_not_nan, 'POBFEM']

print(f'Nuevo Léon tiene un total de {len(df_agebs)} AGEBS urbanas con población.')

Nuevo Léon tiene un total de 2660 AGEBS urbanas con población.


In [152]:
(df_agebs.PNACENT + df_agebs.PNACOE != df_agebs.POBTOT).sum()

2480

In [ ]:
df_agebs.

In [21]:
len(df_mun)

51

Para comenzar vamos a restringir nuestro objetivos a dos variables: edad y sexo. Usemos una AGEB sin datos faltantes, más adelante ajustaremos la implementación para contemplar los datos redactados (*).

Porbaremos con dos AGEBs,  una será la de menor población con datos columnos para las columnas que se refieren a población y sexo, la segunda la de mayor población.

In [213]:
agebs_agg_loc = df_agebs.groupby(['MUN', 'LOC'])[['POBTOT', 'TVIVHAB']].sum()

In [214]:
locs_agg = df_loc.groupby(['MUN', 'LOC'])[['POBTOT', 'TVIVHAB']].sum()

In [215]:
agebs_locs_merge = pd.merge(locs_agg, agebs_agg_loc, left_index=True, right_index=True)
agebs_locs_merge['Agree_POBTOT'] = agebs_locs_merge.POBTOT_x == agebs_locs_merge.POBTOT_y
agebs_locs_merge['Agree_TVIVHAB'] = agebs_locs_merge.TVIVHAB_x == agebs_locs_merge.TVIVHAB_y
agebs_locs_merge

POBTOT_x  TVIVHAB_x  POBTOT_y  TVIVHAB_y  Agree_POBTOT  Agree_TVIVHAB
MUN LOC                                                                       
1   1        1992      544.0      1992      544.0          True           True
2   1        2080      757.0      2080      757.0          True           True
3   1         627      234.0       627      234.0          True           True
4   1       28843     8698.0     28843     8698.0          True           True
5   1       16414     4881.0     16414     4881.0          True           True
6   1      536436   149015.0    536436   149015.0          True           True
    78       8287     2177.0      8287     2177.0          True           True
    133     21180     6012.0     21180     6012.0          True           True
    174      6338     1927.0      6338     1927.0          True           True
    243      7262      750.0      7262      750.0          True           True
    379     11303     2950.0     11303     2950.0          True           True
    385      3717     1016.0      3717     1016.0          True           True
    388     22464     6435.0     22464     6435.0          True           True
    389      7820     2167.0      7820     2167.0          True           True
    392     10113     2658.0     10113     2658.0          True           True
    403      7525     2316.0      7525     2316.0          True           True
    406      4169     1256.0      4169     1256.0          True           True
7   1        2692      748.0      2692      748.0          True           True
    8        2579      805.0      2579      805.0          True           True
8   1        3524     1121.0      3524     1121.0          True           True
9   1       75721    23539.0     75721    23539.0          True           True
    572     29183     8785.0     29183     8785.0          True           True
10  1       11583     3408.0     11583     3408.0          True           True
    129     29920     8328.0     29920     8328.0          True           True
    130     30843     9012.0     30843     9012.0          True           True
    131      9532     2537.0      9532     2537.0          True           True
    132     21275     6343.0     21275     6343.0          True           True
11  1        6739     2212.0      6739     2212.0          True           True
12  1       30217     8794.0     30217     8794.0          True           True
    152      4722     1443.0      4722     1443.0          True           True
    211     30279     9154.0     30279     9154.0          True           True
    228      3048      905.0      3048      905.0          True           True
13  1        8264     2651.0      8264     2651.0          True           True
14  1       11822     2947.0     11822     2947.0          True           True
15  1         802      272.0       802      272.0          True           True
16  1        2210      670.0      2210      670.0          True           True
17  1        7844     2213.0      7844     2213.0          True           True
    153      2543      649.0      2543      649.0          True           True
18  1      234698    66380.0    234698    66380.0          True           True
    25      39998    11725.0     39998    11725.0          True           True
    105     41487    12041.0     41487    12041.0          True           True
    212     53461    16545.0     53461    16545.0          True           True
    234     10649     3133.0     10649     3133.0          True           True
    236      8853     2702.0      8853     2702.0          True           True
19  1      132128    37737.0    132128    37737.0          True           True
20  1        4036     1268.0      4036     1268.0          True           True
21  1      454967   124510.0    454967   124510.0          True           True
    153     17284     4741.0     17284     4741.0          True           True
    155      4076     1153.0      4076     1153.0          

In [219]:
#id_columns = ['ENTIDAD', 'MUN', 'LOC', 'AGEB']

pob_columns = (
    ['POBTOT', 'POBFEM', 'POBMAS']
    + [c for c in df_agebs.columns if c.startswith('P_')]
    + ['POB0_14', 'POB15_64', 'POB65_MAS'])
#df_agebs_pob = df_agebs[id_columns + pob_columns].dropna()
ageb_min = df_agebs[df_agebs.POBTOT == df_agebs_pob.POBTOT.min()].iloc[0]
ageb_max = df_agebs[df_agebs.POBTOT == df_agebs_pob.POBTOT.max()].iloc[0]
#len(df_agebs_pob)

Las columnas de población en la tabla de microdatos serán discretizadas en el menor número intervalos compatibles con las categorías del censo que puedan aprovechar las restricciones.
La ventaja de no usar una única columna de edad con valor entero es la disminución de tamaño del espacio de soluciones.
Imputar una edad específica puede hacerse en una etapa de procesamiento posterior.
Para no contar dos veces soluciones que unicamente difieren en el ordenamiento de las filas, agregaremos restricciones de ordenamiento para los valores de edad.

Las categorias posibles de obtener del censo son las siguientes y las codificamos como enteros:
- 0-2: 0
- 3-4: 1
- 5: 2
- 6-7: 3
- 8-11: 4
- 12-14: 5
- 15-17: 6
- 18-24: 7
- 25-59: 8
- 60-64: 9
- 65-130: 10

Tener esta codificación en mente es importante al momento de definir las restricciones.

Al momento de crear los objetivos, usaremos un arreglo de variables en 2D, la tabla de microdatos.
Esto debe ser compatible con OR-Tools, por lo que será implementado como una lista de listas.
Es este problema restringido a edad y sexo, necesitamos dos columnas: SEXO y EDAD.

In [40]:
def series_to_dict(series):
    d = series.to_dict()
    keys = d.keys()
    
    vals = d.values()
    vals = [int(v) 
            if isinstance(v, float) and not np.isnan(v)
            else v for v in d.values() ]
    
    d = {k: v for k,v in zip(keys, vals)}
    
    return d

## Validando las tablas de contingencia

In [60]:
valid_agebs = df_agebs_pob.apply(validate_ageb, axis=1)

In [61]:
len(df_agebs_pob) - valid_agebs.sum()

427

## Codificando directamente la tabla de contingencia

In [43]:
from ortools.sat.python import cp_model
from itertools import product

In [44]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        #for v in self.__variables:
        #    print('%s=%i' % (v, self.Value(v)), end=' ')
        #print()

    def solution_count(self):
        return self.__solution_count

In [171]:
def create_variables(model, dimensions, N):
    x = {}
    for var_id in product(*dimensions.values()):
        x[var_id] = model.NewIntVar(0, N, f'{var_id}')

    return x

In [173]:
def add_constraint(model, ageb, dimensions, variables, c_name, c_dict, verbose=False):
    assert c_name in ageb.keys()
    
    for k, v in c_dict.items():
        assert k in dimensions.keys()
        for vv in v:
            assert vv in dimensions[k]
    
    # Replace classes with classes involved in the constraint only
    c_dims = dimensions.copy()
    for d in c_dims.keys():
        if d in c_dict.keys():
            c_dims[d] = c_dict[d]
    
    # Find all variables involved in the constraints
    c_vars = []
    for var_id in product(*c_dims.values()):
        c_vars.append(variables[var_id])
    
    if verbose:
        print(' + '.join([v.Name() for v in c_vars]) + f' = {c_name}')
    
    model.Add(sum(c_vars) == ageb[c_name])

In [174]:
model = cp_model.CpModel()

variables = create_variables(model, dimensions, ageb['POBTOT'])

for c_name, c_dict in constraints.items():
    add_constraint(model, ageb, dimensions, variables, c_name, c_dict, verbose=False)
    
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter(list(x.values()))
solver.parameters.enumerate_all_solutions = True

status = solver.Solve(model, solution_printer)
print(f'Found {solution_printer.solution_count()} solutions. Status: {status}')

Found 613872 solutions. Status: 4


In [233]:
ageb_max

ENTIDAD                       19
NOM_ENT               Nuevo León
MUN                           21
NOM_MUN         General Escobedo
LOC                            1
NOM_LOC        Total AGEB urbana
AGEB                        0344
MZA                            0
POBTOT                     11293
POBFEM                    5640.0
POBMAS                    5653.0
P_0A2                      616.0
P_0A2_F                    294.0
P_0A2_M                    322.0
P_3YMAS                  10675.0
P_3YMAS_F                 5344.0
P_3YMAS_M                 5331.0
P_5YMAS                  10157.0
P_5YMAS_F                 5098.0
P_5YMAS_M                 5059.0
P_12YMAS                  8464.0
P_12YMAS_F                4271.0
P_12YMAS_M                4193.0
P_15YMAS                  7948.0
P_15YMAS_F                4005.0
P_15YMAS_M                3943.0
P_18YMAS                  7502.0
P_18YMAS_F                3762.0
P_18YMAS_M                3740.0
P_3A5                      791.0
P_3A5_F   

In [176]:
ageb_max

ENTIDAD            19
MUN                21
LOC                 1
AGEB             0344
POBTOT          11293
POBFEM         5640.0
POBMAS         5653.0
P_0A2           616.0
P_0A2_F         294.0
P_0A2_M         322.0
P_3YMAS       10675.0
P_3YMAS_F      5344.0
P_3YMAS_M      5331.0
P_5YMAS       10157.0
P_5YMAS_F      5098.0
P_5YMAS_M      5059.0
P_12YMAS       8464.0
P_12YMAS_F     4271.0
P_12YMAS_M     4193.0
P_15YMAS       7948.0
P_15YMAS_F     4005.0
P_15YMAS_M     3943.0
P_18YMAS       7502.0
P_18YMAS_F     3762.0
P_18YMAS_M     3740.0
P_3A5           791.0
P_3A5_F         377.0
P_3A5_M         414.0
P_6A11         1420.0
P_6A11_F        696.0
P_6A11_M        724.0
P_8A14         1410.0
P_8A14_F        699.0
P_8A14_M        711.0
P_12A14         516.0
P_12A14_F       266.0
P_12A14_M       250.0
P_15A17         446.0
P_15A17_F       243.0
P_15A17_M       203.0
P_18A24         990.0
P_18A24_F       476.0
P_18A24_M       514.0
P_15A49_F      3337.0
P_60YMAS        426.0
P_60YMAS_F

In [178]:
df_agebs.loc[24218]

ENTIDAD                       19
NOM_ENT               Nuevo León
MUN                           21
NOM_MUN         General Escobedo
LOC                            1
NOM_LOC        Total AGEB urbana
AGEB                        0344
MZA                            0
POBTOT                     11293
POBFEM                    5640.0
POBMAS                    5653.0
P_0A2                      616.0
P_0A2_F                    294.0
P_0A2_M                    322.0
P_3YMAS                  10675.0
P_3YMAS_F                 5344.0
P_3YMAS_M                 5331.0
P_5YMAS                  10157.0
P_5YMAS_F                 5098.0
P_5YMAS_M                 5059.0
P_12YMAS                  8464.0
P_12YMAS_F                4271.0
P_12YMAS_M                4193.0
P_15YMAS                  7948.0
P_15YMAS_F                4005.0
P_15YMAS_M                3943.0
P_18YMAS                  7502.0
P_18YMAS_F                3762.0
P_18YMAS_M                3740.0
P_3A5                      791.0
P_3A5_F   